In [ ]:
!pip install pyodbc
!pip install nltk
!pip install pandas
!pip install textblob
!pip install plotly
!pip install matplotlib
!pip install nbformat

In [1]:
import os
import pyodbc
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from datetime import datetime
from textblob import TextBlob

In [ ]:
# Connect to the SQL Server
connection = (
    "DRIVER={SQL Server};"
    "SERVER=TPBLUE-IST;"                  
    "DATABASE=CustomerFeedbackDB;"         
    "Trusted_Connection=yes;"              
)

# Establish the database connection
dbcon = pyodbc.connect(connection)

# Create a cursor to execute SQL queries
cursor = dbcon.cursor()

# SQL query to fetch feedback text and recipe name
query = """
    SELECT [recipe_name], [feedback_text]
    FROM [CustomerFeedbackDB].[dbo].[ft_user_feedback]
"""

# Execute the query and fetch all results
cursor.execute(query)
feedbacks = cursor.fetchall()

for recipe_name, feedback_text in feedbacks:
    print(recipe_name, feedback_text)

# Close the database connection
dbcon.close()


In [ ]:
# List to store sentiment results
data = []

# Perform sentiment analysis on each feedback
for feedback in feedbacks:
    recipe_name = feedback[0]
    feedback_text = feedback[1]
    blob = TextBlob(feedback_text)
    
    # Get sentiment polarity
    sentiment = blob.sentiment.polarity
    sentiment_Categories = ''
    
    if sentiment > 0 : 
        sentiment_Categories = "Positive"
    elif sentiment < 0 :
         sentiment_Categories = "Negative"
    else:
        sentiment_Categories = "Neutral"
    
    # Append the result as a dictionary
    data.append({
        'recipe_name': recipe_name,
        'feedback_text': feedback_text,
        'sentiment': sentiment_Categories
    })
    
    # Print the results for each feedback
    print(f"Recipe Name: {recipe_name}, Feedback Text: {feedback_text}, Sentiment: {sentiment_Categories}")

# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(data)


In [ ]:
# Get the current timestamp and format it
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Define the output directory and filename with the timestamp
output_dir = r'D:/Data Course/Project'
output_filename = f"Feedback_Analysis_{timestamp}.xlsx"
output_file_path = os.path.join(output_dir, output_filename)

# ---- SUMMARY / PIVOT RESULTS ----
# Create a summary dataframe showing feedback count and sentiment distribution (Positive, Neutral, Negative) by recipe
summary_df = df.pivot_table(
    index='recipe_name', 
    columns='sentiment', 
    aggfunc='size', 
    fill_value=0
).reset_index()

# Rename columns for better readability
summary_df.columns = ['Recipe Name', 'Negative', 'Neutral', 'Positive']

# Add a column for the total number of feedbacks for each recipe
summary_df['Total Feedbacks'] = summary_df[['Positive', 'Neutral', 'Negative']].sum(axis=1)

# Reorder the columns for better presentation
summary_df = summary_df[['Recipe Name', 'Total Feedbacks', 'Positive', 'Neutral', 'Negative']]

# Save the data to an Excel file with multiple sheets
with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
    # Sheet 1: All Results
    df.to_excel(writer, sheet_name='All Results', index=False)
    # Sheet 2: Summarized Results
    summary_df.to_excel(writer, sheet_name='Summarized Results', index=False)

# Print a message to confirm the output file has been saved
print(output_file_path)


In [ ]:
# Calculate total feedback count for each recipe
total_feedbacks = df['recipe_name'].value_counts()

# Get top 10 and bottom 10 recipes by total feedback count
top_10_recipes = total_feedbacks.nlargest(10).index
bottom_10_recipes = total_feedbacks.nsmallest(10).index

# Filter the sentiment counts for top 10 and bottom 10 recipes
top_10_sentiment_counts = df[df['recipe_name'].isin(top_10_recipes)].copy()
bottom_10_sentiment_counts = df[df['recipe_name'].isin(bottom_10_recipes)].copy()

# Reset index to ensure easier plotting and cleaner data manipulation
top_10_sentiment_counts.reset_index(drop=True, inplace=True)
bottom_10_sentiment_counts.reset_index(drop=True, inplace=True)

print("Summary DataFrame:")
print(summary_df)

print("\nTop 10 Recipes Sentiment Counts:")
print(top_10_sentiment_counts)

print("\nBottom 10 Recipes Sentiment Counts:")
print(bottom_10_sentiment_counts)


In [ ]:
# ---- MATPLOTLIB VISUALIZATION: Bar Chart for Top 10 Recipes ----
top_10_sentiment_counts.groupby(['recipe_name', 'sentiment']).size().unstack().plot(kind='bar', stacked=False, color=['red', 'blue', 'green'], ax=plt.gca())

plt.title('Sentiment Distribution by Top 10 Recipes (Matplotlib)')
plt.xlabel('Recipe Name')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.legend(title='Sentiment')
plt.tight_layout()  
plt.show()

# ---- MATPLOTLIB VISUALIZATION: Bar Chart for Bottom 10 Recipes ----
bottom_10_sentiment_counts.groupby(['recipe_name', 'sentiment']).size().unstack().plot(kind='bar', stacked=False, color=['red', 'blue', 'green'], ax=plt.gca())

plt.title('Sentiment Distribution by Bottom 10 Recipes (Matplotlib)')
plt.xlabel('Recipe Name')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.legend(title='Sentiment')
plt.tight_layout()  
plt.show()


In [ ]:
# Group sentiment counts by recipe_name and sentiment for both top 10 and bottom 10
top_10_sentiment_counts = df[df['recipe_name'].isin(top_10_recipes)]
bottom_10_sentiment_counts = df[df['recipe_name'].isin(bottom_10_recipes)]

# ---- PLOTLY VISUALIZATION: Stacked Bar Chart for Top 10 Recipes ----
# Create a bar chart using Plotly
fig_top = px.bar(
    top_10_sentiment_counts,
    x='recipe_name',
    color='sentiment',
    barmode='group',
    title='Sentiment Distribution by Top 10 Recipes (Plotly)',
    labels={'recipe_name': 'Recipe Name', 'count': 'Count'},
    category_orders={'sentiment': ['Negative', 'Neutral', 'Positive']}  
)

# Show the figure
fig_top.show()

# ---- PLOTLY VISUALIZATION: Stacked Bar Chart for Bottom 10 Recipes ----
fig_bottom = px.bar(
    bottom_10_sentiment_counts,
    x='recipe_name',
    color='sentiment',
    barmode='group',
    title='Sentiment Distribution by Bottom 10 Recipes (Plotly)',
    labels={'recipe_name': 'Recipe Name', 'count': 'Count'},
    category_orders={'sentiment': ['Negative', 'Neutral', 'Positive']} 
)

# Show the figure
fig_bottom.show()
